In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
plt.rc('text', usetex=True)
plt.rc('font', family='Serif')

import matplotlib as mpl 
mpl.rcParams['figure.figsize'] = [10, 7]
mpl.rcParams['font.size'] = 27

mpl.rcParams['savefig.dpi'] = 150 
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['xtick.major.size'] = 10 
mpl.rcParams['xtick.minor.size'] = 5 

mpl.rcParams['ytick.major.size'] = 10 
mpl.rcParams['ytick.minor.size'] = 5 

mpl.rcParams['xtick.labelsize'] = 23
mpl.rcParams['ytick.labelsize'] = 23

In [ ]:
NGC3201_path = os.getcwd()

In [ ]:
df = pd.read_csv("{}/NGC3201.dat".format(NGC3201_path),sep='\s+')

In [ ]:
plt.scatter(df['x'],df['y'],s=0.1)

In [ ]:
#determin center
x_guess,y_guess = 3000,3000
radius = 200
def centeroidnp(df):
    fit_stars = df[((df['x']-x_guess)**2 + (df['y']-y_guess)**2) < radius**2]
    length = len(fit_stars)
    x = fit_stars['x'].values
    y = fit_stars['y'].values
    sum_x = np.sum(x)
    sum_y = np.sum(y)
    return sum_x/length, sum_y/length
x_cen, y_cen = centeroidnp(df)
print("X Center is: {}".format(x_cen))
print("Y Center is: {}".format(y_cen))

In [ ]:
#determine MSTO and SGB
plt.scatter(df['VIvega'],df['Vvega'],s=0.5,alpha=1,c='grey')
plt.xlim(0.6,1.0)
plt.ylim(16.5,20)
plt.scatter(0.70,18.3,s=0.7,c='r',label='MSTO')
plt.scatter(0.70+0.05,17.45,s=0.7,c='b',label='SGB')
plt.legend()
plt.gca().invert_yaxis()